In [ ]:
# EfficientNet Fine-Tuning for RFMiD Dataset

In [ ]:
## 🔧 Install dependencies
!pip install -r https://raw.githubusercontent.com/AlexPraxedes12/rfmid_lite/refs/heads/main/requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
## 📂 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

def unzip_and_verify(zip_path, extract_dir, expected_files=[], expected_extensions=[]):
    print(f"📦 Decompressing: {zip_path}")

    # Extract if the folder does not exist
    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall("/content")
        print(f"✅ Dataset uncompressed in: {extract_dir}")
    else:
        print(f"📁 Dataset was already uncompressed in.: {extract_dir}")

    # Verify that the CSV is present if specified
    missing_items = []
    for file in expected_files:
        full_path = os.path.join(extract_dir, file)
        if not os.path.exists(full_path):
            missing_items.append(full_path)

    # Check number of images if desired
    if expected_extensions:
        found_images = []
        for root, _, files in os.walk(extract_dir):
            found_images += [f for f in files if any(f.endswith(ext) for ext in expected_extensions)]
        if not found_images:
            print(f"⚠️ No images with extensions found {expected_extensions} en {extract_dir}.")
        else:
            print(f"🖼️ {len(found_images)} images found in {extract_dir}.")

    # Show missing files
    if missing_items:
        print("❌ Missing files:")
        for m in missing_items:
            print(" -", m)
    else:
        print("✅ All necessary files are present.")

# Training
unzip_and_verify(
    "/content/drive/MyDrive/Training_Set.zip",
    "/content/Training_Set/Training",
    expected_extensions=[".png"]
)

# Validation
unzip_and_verify(
    "/content/drive/MyDrive/Evaluation_Set.zip",
    "/content/Validation_Set/Validation",
    expected_files=["RFMiD_Validation_Labels.csv"],
    expected_extensions=[".png"]
)

# Test
unzip_and_verify(
    "/content/drive/MyDrive/Test_Set.zip",
    "/content/Test_Set/Test",
    expected_files=["RFMiD_Testing_Labels.csv"],
    expected_extensions=[".png"]
)


📦 Descomprimiendo: /content/drive/MyDrive/Training_Set.zip
✅ Dataset descomprimido en: /content/Training_Set/Training
🖼️ 1920 imágenes encontradas en /content/Training_Set/Training.
✅ Todos los archivos necesarios están presentes.
📦 Descomprimiendo: /content/drive/MyDrive/Evaluation_Set.zip
✅ Dataset descomprimido en: /content/Validation_Set/Validation
⚠️ No se encontraron imágenes con extensiones ['.png'] en /content/Validation_Set/Validation.
❌ Archivos faltantes:
 - /content/Validation_Set/Validation/RFMiD_Validation_Labels.csv
📦 Descomprimiendo: /content/drive/MyDrive/Test_Set.zip
✅ Dataset descomprimido en: /content/Test_Set/Test
🖼️ 640 imágenes encontradas en /content/Test_Set/Test.
❌ Archivos faltantes:
 - /content/Test_Set/Test/RFMiD_Testing_Labels.csv


In [ ]:
## 📜 Download training script
!wget -N https://raw.githubusercontent.com/AlexPraxedes12/rfmid_lite/refs/heads/main/train_efficientnet.py


--2025-07-11 21:50:06--  https://raw.githubusercontent.com/AlexPraxedes12/rfmid_lite/refs/heads/main/train_efficientnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8086 (7.9K) [text/plain]
Saving to: ‘train_efficientnet.py’

train_efficientnet. 100%[===================>]   7.90K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-07-11 21:50:06 (111 MB/s) - ‘train_efficientnet.py’ saved [8086/8086]



In [ ]:
## 🚀 Run training
!python train_efficientnet.py

Loaded labels with shape (1920, 29)
Loaded labels with shape (640, 29)
Loaded labels with shape (640, 29)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[Epoch 1] Training Loss: 0.2814
[Epoch 1] Validation Loss: 0.1287
Saved best model
[Epoch 2] Training Loss: 0.1133
[Epoch 2] Validation Loss: 0.1055
Saved best model
[Epoch 3] Training Loss: 0.0958
[Epoch 3] Validation Loss: 0.0959
Saved best model
[Epoch 4

In [ ]:
import shutil

# Make sure you have mounted Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the model to your Drive folder
shutil.copy('/content/efficientnet_rfmid_best.pth', '/content/drive/MyDrive/efficientnet_rfmid_best.pth')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/efficientnet_rfmid_best.pth'